# Facebook: Paid vs Unpaid Posts

### Introduction:
Background: 
    We are in the digital era, and Facebook is one of the biggest social media platforms known to society. Hence, it is a great place for businesses to reach consumers globally. With this idea in mind, we believe it would be beneficial for businesses to know if they should pay for a post to achieve a certain number of post total reach, post engagements, and page likes, or if it is better to follow the organic route. This is significant for designing marketing strategies, especially for small businesses or start-ups with small funds. We chose two different metrics for Facebook posts that we thought would have the highest influence on the success of advertising, lifetime post total reach and lifetime people who have liked the page and engaged with the post, to make a prediction on whether the post was paid or unpaid.

Question: 
    **Can we use the amount of lifetime post total reach and lifetime people who have liked the page and engaged with the post to predict whether a future post is paid or unpaid?**

Dataset: 
    The [dataset](https://archive.ics.uci.edu/ml/machine-learning-databases/00368/Facebook_metrics.zip) we have is from 790Facebook posts of a renowned cosmetic brand in 2014, and it shows the type of post (paid vs unpaid) as well as information on it, from the number of likes and shares to total impressions. According to the paper from which the dataset is based on, "Most of the information was exported directly from the company's Facebook page", and this includes "paid", "lifetime post total reach", and "lifetime people who have liked the page and engaged with the post", the columns that we are interested in. The link to the dataset is: https://archive.ics.uci.edu/ml/machine-learning-databases/00368/Facebook_metrics.zip 

Predictor variable description: 

   "Lifetime post total reach" is a chosen predictor because it is a measure of how much exposure a post achieves by the quantifying the number of unique users who have seen the post in their screen. In this scenario, "unique users" mean that an individual who owns multiple accounts is only considered as one user. 

   On the other hand, "lifetime people who have liked the page and engaged with the post" is the number of unique users who have liked the page and clicked anywhere in a page's post. "Engagements" pertain to all the types and origins of clicks on the specific post. This predictor is chosen as it is said to be a stronger measure for user feedback, since this would ensure that the user is interested in the post. Therefore, we use these two predictors to determine if the post was organic or not because we believe companies would want to know whether their post is both seen and engaged with. (http://www.math-evry.cnrs.fr/_media/members/aguilloux/enseignements/m1mint/moro2016.pdf)

### Methods and Results

##### Step 1
The following packages are loaded using the `library` function to be able to use the functions necessary to build, tune, and estimate the accuracy of the classifier.

In [1]:
# load packages
library(repr)
library(tidyverse)
library(tidymodels)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

##### Step 2
The dataset can be downloaded from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Facebook+metrics). The as_factor() function is used to convert the Paid column into a factor instead of an integer. With this, it can now be used as a class/label that observations can be categorized into. Moreover, the select() function is used to select the columns, Lifetime Post Total Reach, and Lifetime People who have liked your Page and engaged with your post, as the chosen predictors for our classifier. Furthermore, in this dataset, 0 = unpaid and 1 = paid.

In [6]:
set.seed(3060) # setting seed to make data analysis reproducible

options(repr.plot.width = 8, repr.plot.height = 8)
options(repr.matrix.max.rows = 6)

# load data
temp <- tempfile()
download.file("https://archive.ics.uci.edu/ml/machine-learning-databases/00368/Facebook_metrics.zip", temp, mode = "wb")
unzip(temp, "dataset_Facebook.csv")
facebook_unscaled <- read.delim("dataset_Facebook.csv", sep = ";") %>% 
    mutate(Paid = as_factor(Paid)) %>% 
    select(Paid, 
           Lifetime_Post = Lifetime.Post.Total.Reach,
           Lifetime_Like_Engage = Lifetime.People.who.have.liked.your.Page.and.engaged.with.your.post
          )

# scale data
facebook <- facebook_unscaled %>%
    filter(!is.na(Paid)) %>%
    mutate(Lifetime_Post = scale(Lifetime_Post, center = TRUE), Lifetime_Like_Engage = scale(Lifetime_Like_Engage, center = TRUE))
facebook

Paid,Lifetime_Post,Lifetime_Like_Engage
<fct>,"<dbl[,1]>","<dbl[,1]>"
0,-0.4908219,-0.8016572
0,-0.1521492,0.8111968
0,-0.5057168,-0.7804569
⋮,⋮,⋮
0,-0.4588352,-0.5048529
0,-0.4457417,-0.4037438
0,-0.4291332,-0.3923282


**Table 1: Scaled Facebook Dataset**

##### Step 3
The dataset is then randomly shuffled and split into a training and testing set, wherein the training set consists of 75% of the data and the testing set consists of the remaining 25%. The training set will be used to build and tune the k-nn classifier, and the testing set will be used to estimate the prediction accuracy of this classifier. We also specify the classifier that the class we want to predict is the Paid column.

In [ ]:
# splitting the data into a training and testing dataset
facebook_split <- initial_split(facebook, prop = 0.75, strata = Paid) 
facebook_train <- training(facebook_split)   
facebook_test <- testing(facebook_split)

##### Step 4
Using the training dataset, we build and tune the classifier that we will use to predict observations. It is important that we scale and center the chosen predictors, using the setp_scale() and step_center() functions, to ensure that no variable would have a higher influence in the prediction due to their larger scale. We include in our recipe that the predictors we want are Lifetime_Post and Liketime_Like_Engage, therefore using all_predictors() would mean that these are the variables we would like to scale and center. Furthermore, we do 5-fold cross-validation, using the vfold_cv() function, to build the model to establish the number of neighbours that would be best to achieve the highest estimated accuracy. We then add the recipe and model into our workflow wherein we tell the classifier we want to try tuning it with 10 different Ks through the grid argument.

In [ ]:
#building and tuning the classifier
facebook_recipe <- recipe(Paid ~ Lifetime_Post + Lifetime_Like_Engage, data = facebook_train) %>%
   step_scale(all_predictors()) %>%
   step_center(all_predictors())

facebook_vfold <- vfold_cv(facebook_train, v = 5, strata = Paid)

knn_tune<- nearest_neighbor(weight_func = 'rectangular', neighbors = tune()) %>%
      set_engine('kknn') %>%
      set_mode('classification')

knn_results <- workflow() %>%
      add_recipe(facebook_recipe) %>%
      add_model(knn_tune) %>%
      tune_grid(resamples = facebook_vfold, grid = 10) %>%
      collect_metrics()

##### Step 5
Because we only care about the mean of the accuracy, we use the filter() function to filter out these specific rows under the .metric column. We then plot out the graph wherein the x-axis consists of the neighbors column and the y-axis contains the respective means of these neighbours. 

In [ ]:
#choosing the correct number of neighbours
accuracies <- knn_results %>% 
      filter(.metric == 'accuracy')

accuracy_versus_k <- ggplot(accuracies, aes(x = neighbors, y = mean))+
      geom_point() +
      geom_line() +
      labs(x = "Neighbors", y = "Accuracy Estimate")

accuracy_versus_k

**Graph 1: K value vs Accuracy Graph**

##### Step 6
Through the graph, we see that the number of neighbours with the highest accuracy estimate of around 70% is 14. We use the filter() function to filter out the row with the highest mean and pull(neighbors) function to pull the value of the 

In [ ]:
#building classifier with chosen number of neighbours
k_max <- accuracies %>%
    filter(mean == max(mean)) %>%
    pull(neighbors)
k_max

knn_spec <- nearest_neighbor(weight_func = 'rectangular', neighbors = k_max) %>%
      set_engine('kknn') %>%
      set_mode('classification')

In [ ]:
#estimating the classifier's validation/prediction accuracy
facebook_fit <- workflow() %>%
      add_recipe(facebook_recipe) %>%
      add_model(knn_spec) %>%
      fit_resamples(resamples = facebook_vfold)

facebook_metrics<-collect_metrics(facebook_fit)
facebook_metrics

**Table 2: Classifier's Prediction Accuracy on the Training Dataset**

In [ ]:
#estimating the accuracy of the classifier on the testing data
facebook_fit_test <- workflow() %>%
    add_recipe(facebook_recipe) %>%
    add_model(knn_spec) %>%
    fit(data = facebook_train)

facebook_test_prediction <- predict(facebook_fit_test, facebook_test) %>%
    bind_cols(facebook_test)

facebook_metrics <- facebook_test_prediction %>%
    metrics(truth = Paid, estimate = .pred_class)
facebook_metrics

**Table 3: Classifier's Prediction Accuracy on the Testing Dataset**

In [ ]:
confusion <- facebook_test_prediction %>%
    conf_mat(truth = Paid, estimate = .pred_class)
confusion

**Table 4: Confusion Matrix**

Description of predictor variables relationship:

In [ ]:
#plotting the scaled variables
facebook_plot <- ggplot(facebook, aes(x = Lifetime_Post, y = Lifetime_Like_Engage, colour = Paid)) +
    geom_point() +
    labs(x = "Total Reach of Post (Lifetime)", y = "Number of People Who Have Liked Page \n and Engaged with  Post (Lifetime)",
              color = "Post Type") +
    theme(text = element_text(size = 20))
facebook_plot

**Graph 2: Lifetime Post Total Reach vs Lifetime People who have Liked the page and Engaged with the post**

### Discussion:

Overall, it was found that Lifetime Post reach and Lifetime Likes and Engagement are moderately accurate predictors of whether a Facebook post is paid or not. Although the classifiers have a combined accuracy of around 70%, the predicted trend is not reliably seen in our results. Contrary to our hypothesis, a higher lifetime post count and higher lifetime likes + engagement did not reliably correlate to a paid post. This finding seemingly replicates the findings of another study 
(https://www.sciencedirect.com/science/article/pii/S0148296316000813), where it was found that a paid post only boosts its reachability by 7%. Interestingly, lifetime post reach is a slightly more accurate predictor, compared to the other variable we tested. Another study by (https://www.tandfonline.com/doi/abs/10.1080/10641734.2018.1503113?journalCode=ujci20) found that many different factors may affect post likes and engagement. For example, post content and even when the post was created have significant effects on how many likes/engagements the post achieves. Thus, these other factors should be examined in future studies to determine how they may act as moderating factors. In terms of application, companies can use this information, combined with actual purchasing data, to streamline their advertising strategies.  Specifically, paying for a specific type of post in a specific month may help increase post reach, which would increase potential consumer exposure, which may in turn increase purchases.

### Reference: